In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re #regular expressions
import plotly.express as px
# Plotting
import matplotlib.pylab as plt
import seaborn as sns

#Progress bar
from tqdm import tqdm
from datetime import datetime
import os
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#upload the data
path = '../input/novel-corona-virus-2019-dataset/covid_19_data.csv'

In [ ]:
#loading datasets
covid_df = pd.read_csv(path)

In [ ]:
#Split the date column into month and year then add those two columns

covid_df.ObservationDate = covid_df.ObservationDate.apply(lambda x: x.replace('/', '-'))
covid_df['observation_date']= pd.to_datetime(covid_df['ObservationDate']) 

#coverting last update into datetime



covid_df['last_update'] = covid_df['Last Update'].apply(lambda x: x.replace('/', '-'))
covid_df['last_update']= pd.to_datetime(covid_df['Last Update']) 

covid_df.head()


In [ ]:
# last_update, timestamp = zip(*[(d.date(), d.time()) for d in covid_df['last_update']])
# covid_df = covid_df.assign(last_update=last_update, update_time=timestamp)
# covid_df.head()

In [ ]:
covid_df.dtypes


In [ ]:
covid_df['last_update']= pd.to_datetime(covid_df['last_update']) 

covid_df.dtypes


In [ ]:
covid_df=covid_df.drop(['Province/State','Last Update', 'ObservationDate'], 1, inplace= False)

In [ ]:
#lower case
covid_df.columns = map(str.lower, covid_df.columns)
covid_df.columns

# Modeling

In [ ]:
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error

from fbprophet.plot import plot_plotly

In [ ]:
covid_train=pd.DataFrame(covid_df.groupby('last_update')['deaths'].sum())
covid_one=pd.DataFrame(covid_df.groupby('last_update')['deaths', 'confirmed', 'recovered'].sum())

covid_train.shape
covid_one.head()


In [ ]:
covid_train=covid_train.reset_index() \
              .rename(columns={'last_update':'ds',
                               'deaths':'y'})
covid_train.head()

In [ ]:
# Setup and train model and fit
from fbprophet import Prophet
model = Prophet()
model.fit(covid_train.reset_index() \
              .rename(columns={'last_update':'ds',
                               'deaths':'y'}))

future = model.make_future_dataframe(periods=365)
recoverd_predicted = model.predict(future)
future.tail()

In [ ]:
# Predict on training set with model
covid_test_fcst = model.predict(df=covid_train.reset_index() \
                                   .rename(columns={'last_update':'ds'}))

In [ ]:
covid_test_fcst.head()

In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
fig = model.plot(covid_test_fcst,
                 ax=ax)
plt.show()

In [ ]:
future = model.make_future_dataframe(periods=365)
fatalities_predict = model.predict(future)

In [ ]:
fig = plot_plotly(model, fatalities_predict)  
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Predictions : Covid 19 confirmed cases',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations)
fig

In [ ]:
# Plot the components of the model
fig = model.plot_components(covid_test_fcst)

In [ ]:
mse=mean_squared_error(y_true=covid_train['y'],
                   y_pred=covid_test_fcst['yhat'])
mse

In [ ]:
#Root means squared error
from math import sqrt
rmse=sqrt(mse)
rmse


## Evaluation

In [ ]:
covid_train.shape

In [ ]:
m = Prophet()
m.fit(covid_train.reset_index() \
              .rename(columns={'last_update':'ds',
                               'deaths':'y'}))
future = m.make_future_dataframe(periods=366)

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(m, initial='53 days', period='366 days', horizon = '22 days')
df_cv

In [ ]:
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='mape')
fig

## Neural networks

## Feature Engineering

### Creating dataframes for recovered and confirmed cases.

In [ ]:
covid_two=pd.DataFrame(covid_df.groupby('last_update')['confirmed'].sum())

covid_two.head()

In [ ]:
covid_two=covid_two.reset_index() \
              .rename(columns={'last_update':'ds',
                               'confirmed':'c'})
covid_two.head()

In [ ]:
covid_one=pd.DataFrame(covid_df.groupby('last_update')['recovered'].sum())

covid_one.head()

In [ ]:
covid_one=covid_one.reset_index() \
              .rename(columns={'last_update':'ds',
                               'recovered':'r'})
covid_one.head()

### Dropping the date column

In [ ]:
covid_one=covid_one.drop(['ds'], 1, inplace=False)
covid_two=covid_two.drop(['ds'], 1, inplace=False)


In [ ]:
covid=covid_train.drop(['ds'], 1, inplace=False)
covid.head()

In [ ]:
covid.shape

In [ ]:
covid_one.shape

### Joining the confirmed and recovered cases dataset

In [ ]:
covid_new = covid_one.join(covid_two)
covid_new.head()

## Model

In [ ]:
X= covid_new
Y=covid

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Import an Multi-Layer Perceptron Regressor model estimator from Scikit-Learn's neural_network library
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
from sklearn import metrics

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale, Y, test_size=0.3)

In [ ]:
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

In [ ]:
print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(2,)),
    Dense(32, activation='relu'),
    Dense(1, activation='relu'),
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, Y_train,
          batch_size=32, epochs=7,
          validation_data=(X_val, Y_val))

In [ ]:
model.evaluate(X_test, Y_test)[1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()